# Phase 3: RNN

## Import các thư viện cần thiết

In [101]:
import json
import numpy as np
import pandas as pd

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import tensorflow as tf

from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
import time
import psutil
import os

## Đọc dữ liệu huấn luyện và dữ liệu test

In [102]:
# Đường dẫn đến thư mục chứa dữ liệu
path_to_data='.'

In [103]:
# Đọc dữ liệu huấn luyện từ file JSON
with open(path_to_data+'/train.json','r') as f:
    train_data = json.load(f)

# Đọc dữ liệu test từ file JSON
with open(path_to_data+'/test.json','r') as f:
    test_data = json.load(f)

# Chuyển  dữ liệu thành DataFrame để dễ quan sát
train_df = pd.DataFrame(train_data)
test_df = pd.DataFrame(test_data)

In [104]:
# In ra dữ liệu huấn luyện
train_df.head()

,audio_embedding,is_turkey,vid_id,end_time_seconds_youtube_clip,start_time_seconds_youtube_clip
0,"[[172, 34, 216, 110, 208, 46, 95, 66, 161, 125...",0,kDCk3hLIVXo,70,60
1,"[[169, 20, 165, 102, 205, 62, 110, 103, 211, 1...",1,DPcGzqHoo7Y,40,30
2,"[[148, 8, 138, 60, 237, 48, 121, 108, 145, 177...",1,7yM63MTHh5k,240,230
3,"[[151, 0, 162, 88, 171, 71, 47, 90, 179, 190, ...",1,luG3RmUAxxM,520,510
4,"[[162, 17, 187, 111, 211, 105, 92, 67, 203, 15...",0,PIm3cjxTpOk,10,0


In [105]:
# In ra dữ liệu kiểm tra
test_df.head()

,audio_embedding,vid_id,end_time_seconds_youtube_clip,start_time_seconds_youtube_clip
0,"[[177, 20, 226, 132, 198, 81, 111, 59, 132, 18...",pyKh38FXD3E,10,0
1,"[[169, 21, 204, 161, 195, 72, 60, 39, 152, 184...",THhP1idrWXA,40,30
2,"[[165, 13, 198, 141, 199, 81, 173, 54, 119, 11...",jsw3T6GY2Nw,40,30
3,"[[167, 18, 188, 159, 198, 63, 156, 36, 179, 22...",nFkXTMHcjMU,24,14
4,"[[178, 32, 181, 100, 198, 46, 82, 83, 136, 227...",Au8g9kAlrLQ,40,30


## Phát triển mô hình

### Xử lý dữ liệu mô hình trước khi huấn luyện

In [106]:
# Hàm để padding các embeddings về cùng kích thước
def padding_embeddings(embeddings,max_frames=10,frame_size=128):
    # Nếu số frame nhỏ hơn max_frames → pad thêm frame 0
    if len(embeddings) < max_frames:
        pad_len = max_frames - len(embeddings)
        padding = [[0] * frame_size] * pad_len
        embeddings = embeddings + padding
    # Nếu số frame lớn hơn → cắt bớt
    elif len(embeddings) > max_frames:
        embeddings = embeddings[:max_frames]
    return np.array(embeddings)

### Chuyển đổi dữ liệu thành mảng numpy và dùng padding

In [107]:
# Chuyển đổi dữ liệu huấn luyện và test thành mảng NumPy với padding
train_X = np.array([padding_embeddings(item['audio_embedding']) for item in train_data])
train_Y = np.array([item['is_turkey'] for item in train_data])
valid_idx = test_df['audio_embedding'].apply(lambda x: isinstance(x, list) and len(x) > 0)
test_X = np.array([padding_embeddings(item['audio_embedding']) for item in test_data])

### Chuẩn hóa dữ liệu

In [108]:
scaler = StandardScaler()
X_train = scaler.fit_transform(train_X.reshape(-1, 128)).reshape(-1, 10, 128) # Chuyển đổi train_X về dạng 2D cho scaler, sau đó reshape lại về dạng 3D
X_test = scaler.transform(test_X.reshape(-1, 128)).reshape(-1, 10, 128) # Chuyển đổi test_X về dạng 2D cho scaler, sau đó reshape lại về dạng 3D
X_train, X_val, y_train, y_val = train_test_split(train_X, train_Y, test_size=0.2, random_state=45) # Chia dữ liệu thành tập huấn luyện và tập 


### Xây dựng mô hình

In [109]:
model=models.Sequential([
    layers.Input(shape=(10, 128)), # Đầu vào
    layers.Masking(mask_value=0.0), # Bỏ qua các giá trị padding
    layers.SpatialDropout1D(0.1), # Giảm overfitting bằng cách dropout theo không gian
    layers.LSTM(128,return_sequences=False), # LSTM với 128 units
    layers.Dropout(0.1), # Dropout để giảm overfitting
    layers.Dense(32,activation='relu'), # Dense layer với 32 units và ReLU activation
    layers.Dropout(0.1), # Dropout để giảm overfitting
    layers.Dense(1, activation='sigmoid') # Đầu ra với sigmoid activation
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['auc']) # Biên dịch mô hình với hàm mất mát binary_crossentropy và metric AUC

### Xây dựng callback

In [110]:
early_stop= [EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True), # Dừng sớm nếu không cải thiện trong 5 epoch
            ReduceLROnPlateau(monitor='val_loss', patience=3, factor=0.5, verbose=1)] # Giảm learning rate nếu không cải thiện trong 3 epoch

### Huấn luyện mô hình

In [111]:

time_used=0 # Thời gian sử dụng
mem_used=0 # Bộ nhớ sử dụng
process = psutil.Process(os.getpid())
mem_before = process.memory_info().rss / (1024 ** 2)  # MB # Lấy bộ nhớ trước khi huấn luyện

start_time = time.time() # Bắt đầu thời gian huấn luyện
history = model.fit(X_train, y_train, epochs=50, batch_size=64, validation_data=(X_val, y_val), callbacks=[early_stop]) # Huấn luyện mô hình với dữ liệu huấn luyện và validation
end_time = time.time() # Kết thúc thời gian huấn luyện
mem_after = process.memory_info().rss / (1024 ** 2)  # MB # Lấy bộ nhớ sau khi huấn luyện
mem_used += (mem_after - mem_before) # Tính bộ nhớ sử dụng
time_used += (end_time - start_time) # Tính thời gian sử dụng

Epoch 1/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 3s 51ms/step - auc: 0.6293 - loss: 0.6708 - val_auc: 0.8865 - val_loss: 0.5196 - learning_rate: 0.0010
Epoch 2/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - auc: 0.8847 - loss: 0.4787 - val_auc: 0.9409 - val_loss: 0.3757 - learning_rate: 0.0010
Epoch 3/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - auc: 0.9243 - loss: 0.3830 - val_auc: 0.9486 - val_loss: 0.3323 - learning_rate: 0.0010
Epoch 4/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - auc: 0.9467 - loss: 0.3314 - val_auc: 0.9456 - val_loss: 0.3203 - learning_rate: 0.0010
Epoch 5/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - auc: 0.9272 - loss: 0.3490 - val_auc: 0.9644 - val_loss: 0.2847 - learning_rate: 0.0010
Epoch 6/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - auc: 0.9419 - loss: 0.3129 - val_auc: 0.9627 - val_loss: 0.2739 - learning_rate: 0.0010
Epoch 7/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - auc: 0.9351 - loss: 0.3273 - val_auc: 0.9731 - val_loss: 0.2444 - learning_rate: 0.0010
Epoch 8/50
15

## ĐÁNH GIÁ MÔ HÌNH

### Đánh giá mô hình trên tập validation

In [112]:
y_pred_prod = model.predict(X_val).ravel() # Dự đoán xác suất cho tập validation
y_pred= (y_pred_prod > 0.5).astype(int) # Chuyển đổi xác suất thành nhãn 0 hoặc 1

auc= roc_auc_score(y_val, y_pred_prod) # Tính AUC
accuracy = accuracy_score(y_val, y_pred) # Tính accuracy
precision = precision_score(y_val, y_pred) # Tính precision
recall = recall_score(y_val, y_pred) # Tính recall
f1 = f1_score(y_val, y_pred) # Tính F1 score

print(f"AUC: {auc:.4f}")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Time used: {time_used:.2f} seconds")
print(f"Memory used: {mem_used:.2f} MB")

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
AUC: 0.9902
Accuracy: 0.9582
Precision: 0.9505
Recall: 0.9505
F1 Score: 0.9505
Time used: 13.64 seconds
Memory used: 36.53 MB


### Dự đoán trên tập test

In [113]:
test_pred_prod = model.predict(test_X).ravel() # Dự đoán xác suất cho tập test

# Lưu kết quả dự đoán vào file CSV
test_df['is_turkey'] = -1.0
test_df.loc[valid_idx, 'is_turkey'] = test_pred_prod
test_df.loc[valid_idx, ['vid_id', 'is_turkey']].to_csv('result.csv', index=False)

38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
